# LEPL1106, Devoir 4 : série de Fourier

## 0) Introduction

*Actuellement en pleine préparation de son prochain album, "Step Kronecker Impulsion", le rappeur 20 Cent est débordé et a besoin de votre aide. En effet, le beat d'une de ses chansons ne lui convient pas ; il souhaite booster les basses fréquences (fréquences graves) pour donner plus de "volume" au son. Pour cela, il souhaite dans un premier temps analyser le contenu fréquentiel du signal audio de son beat, pour ensuite, à partir de cette représentation, augmenter les composantes du signal associées aux basses fréquences. Il souhaite donc que vous l'aidiez à calculer les coefficients en série de Fourier d'un signal périodique.*

Pour rappel, la série de Fourier (FS) d'un signal discret périodique $x[n]$ de période fondamentale $N$ est donnée par :

$$X[k] = \frac{1}{N}\sum_{n=0}^{N-1} x[n] e^{-jk\Omega_0n} \qquad \qquad k\in 0,1,...,N-1$$

où $\Omega_0 = \frac{2\pi}{N}$. Les coefficients $X[k]$ obtenus sont également périodiques, de période fondamentale $N$ (c'est-à-dire, $X[k+N] = X[k]$, *vérifiez-le !*)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Tailles de police utilisées avec la librairie matplotlib
legendsize = 14
titlesize = 14
labelsize = 16

## 1) Transformée de Fourier discrète (DFT en anglais) et Fast Fourier Transform (FFT)

Sans surprise, il existe une fonction intégrée à [NumPy](http://www.numpy.org/) qui réalise "presque" le calcul de la série de Fourier : il s'agit de la fonction [FFT](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html). Voyons de suite son docstring :

In [ ]:
help(np.fft.fft) #Notez que la fonction fft se trouve dans le module fft de numpy, d'où np.fft.fft

Sur base du docstring, on constate que la fonction `np.fft.fft` calcule la transformée de Fourier discrète, surnommée DFT pour *Discrete Fourier Transform* en anglais (à ne pas confondre avec la DTFT - *Discrete Time Fourier Transform* !). Pour ce faire, elle utilise l'algorithme *Fast Fourier Transform*$^1$ (FFT). Pour éviter toute confusion : la FFT est donc une *implémentation possible* (particulièrement efficace) de la DFT.


*$^1$ Pour les curieux, voir [Wikipédia](https://en.wikipedia.org/wiki/Fast_Fourier_transform) mais cela sort largement du cadre du cours. Retenez simplement que grâce à l'algorithme FFT, on peut calculer une DFT très rapidement _lorsque la longueur $N$ du signal fourni en entrée est une puissance de 2_*.


> Oui mais c'est quoi une DFT ?
>
> -- étudiant anonyme qui pensait avoir tout vu avec la séance 4

Une DFT permet d'évaluer numériquement les transformées (=/= séries) de Fourier. Sans rentrer dans trop de détails pour le moment, sachez que la définition de la DFT pour un signal $x[n]$ tronqué à $N$ échantillons est la suivante :

$$X_{DFT}[k] = \sum_{n=0}^{N-1} x[n] e^{-j2\pi kn/N} \qquad \qquad k\in 0,1,...,N-1$$

La DFT est aussi périodique, de période $N$ (soit $X_{DFT}[k+N]=X_{DFT}[k]$).

> Je me sens perdu avec toutes ces notions de transformées et séries !
>
> -- étudiant anonyme

Voici une petite synthèse de ces concepts, qui sont généralement utilisés pour passer de la représentation en domaine direct d'un signal à sa représentation en domaine fréquentiel :
 - FS (continu) : La Série de Fourier en temps continu ($x(t) \Rightarrow X[k]$) part d'un signal continu périodique;
 - FS (discret) : La Série de Fourier en temps discret ($x[n] \Rightarrow X[k]$) part d'un signal discret périodique;
 - FT : La Transformée de Fourier ($x(t) \Rightarrow X[j\omega]$) part d'un signal continu non-périodique;
 - DTFT : La Transformée de Fourier en temps discret ($x[n] \Rightarrow X(e^{j\Omega})$) part d'un signal discret non-périodique;
 - DFT : La Transformée de Fourier discrète ($x[n] \Rightarrow X[k]$) part d'un signal discret à support fini;
 - FFT : Il s'agit d'une implémentation possible pour calculer efficacement la DFT


Le concept de la DFT sera abordé lors du cours et une séance d'exercice complète lui est dédiée. Pas de panique si vous ne saisissez pas encore toutes les subtilités ! $^2$

Bref, outre un facteur $\frac{1}{N}$, DFT et FS (discret) semblent similaires pour des signaux discrets. La différence fondamentale réside dans l'interprétation implicite du $x[n]$ : 

- Pour la DFT, le signal $x[n]$ est un signal à support fini, avec $N$ échantillons. 

- Pour la FS, le signal $x[n]$ est périodique, de période $N$, et se répète donc en dehors des $N$ échantillons donnés à la FS. Généralement, on utilise une période du signal $x[n]$ pour calculer sa FS.

> Et donc je dois faire quoi pour mon devoir ???
>
> -- étudiant anonyme qui commence doucement à s'inquiéter

On va se restreindre ici à des signaux périodiques, de période $N$. On peut alors calculer leurs coefficients en série de Fourier (FS) simplement en calculant la DFT de taille $N$ (implémentée par l'*algorithme* FFT) et en corrigeant le facteur $\frac{1}{N}$ manquant dans la définition de la DFT par rapport à celle de la FS. 

*$^2$ Pour celles et ceux qui voudraient mieux comprendre ou en savoir plus, n'hésitez pas à vous orienter vers le forum Moodle, votre tutrice/tuteur ou le reste de l'équipe encadrante.*

## 2) Calcul des coefficients de la série de Fourier d'un signal périodique

### Signal x[n]

Commençons par définir un petit cosinus de période $N=2$, auquel on somme une constante :

$$ x[n] = 2 + \cos(n \pi) $$

> Tiens donc, je me rappelle de ce signal... J'ai dû calculer les coefficients de sa série de Fourier lors de l'exercice 4.6 b) de la séance 4. Je vais donc pouvoir vérifier la réponse que j'obtiens ici avec celle que j'ai calculée en séance...
>
> -- étudiant anonyme plutôt malin

Petit test rapide pour voir à quoi ressemble notre signal :

In [ ]:
N = 2
n = np.arange(0,5*N)
x = 2 + np.cos(n*np.pi)

print("n : ",n)
print("x : ",x)

plt.figure(figsize=(12,6))
markerline, stemlines, baseline = plt.stem(n,x)
plt.setp(baseline,color='k')
plt.setp(stemlines,color='b',linewidth=2,linestyle='--')
plt.setp(markerline,color='b',marker='o',markersize=10)
plt.title(f"Signal $x[n]$",fontsize=titlesize)
plt.xlabel('$n$ [-]',fontsize=labelsize)
plt.ylabel('$x[n]$ [-]',fontsize=labelsize)
plt.grid('on',alpha=0.3)
plt.ylim((-0.2,3.5))
plt.show()

Comme attendu, le signal est bien périodique, de période $N=2$.

### Avez-vous bien suivi l'introduction théorique ?

Nous vous proposons de commencer le devoir par vous poser les questions suivantes, sur base de l'introduction théorique ci-dessus...

Le signal $x[n]$ est-il...
 - Discret ?
 - Continu ?
 - Périodique ?
 - Non-périodique ?
 
Après calcul des coefficients de Série de Fourier, le signal $X[k]$ sera-t-il...
 - Discret ?
 - Continu ?
 - Périodique ?
 - Non-périodique ?
 - Obtenu à l'aide d'une DTFT ?
 
Ce même signal $X[k]$ est-il une fonction...
 - Paire ?
 - Impaire ?
 - Ni paire ni impaire ?
 
Les réponses à ces questions devront être données sur INGInious.

### Calcul des coefficients de la série de Fourier

A vous d'implémenter maintenant la fonction `serieFourier(x,N)`, calculant les coefficients de la série de Fourier d'un signal périodique $x[n]$. 

L'argument $N$ est un entier et correspond à la période *fondamentale* du signal. L'argument $x$ est un vecteur de longueur $pN$ où $p\in \mathbb N$ et dont les entrées sont des valeurs réelles. Le support du signal $x$ est donc $\{0,\dots,pN-1\}$.

La fonction doit renvoyer deux vecteurs $X$ et $k$ décrits ci-dessous (attention à bien retourner le tuple $(X,k)$).
Le vecteur $X$, __de longueur $N$__, contient les coefficients de Fourier du signal périodique $x[n]$ __calculés sur une seule période__, ceux-ci pouvant être complexes. Le vecteur $k$ contient les indices (entiers relatifs) correspondant aux coefficients  $X$ de la série de Fourier. 

Attention, on vous demande en outre de :

- supprimer les erreurs d'arrondis en remplaçant par 0 les coefficients $X[k]$ ayant une **amplitude en valeur absolue** inférieure à 1e-9

- centrer les coefficients $X[k]$ obtenus autour de $k=0$ (ce qui correspond à la fréquence nulle). Pour ce faire, utilisez la fonction `np.fft.fftshift` (voir [ici](https://numpy.org/doc/stable/reference/generated/numpy.fft.fftshift.html)), qui s'utilise directement sur la sortie de la fonction `np.fft.fft`. Après avoir appliqué un `np.fft.fftshift`, le vecteur des $k$ ne correspond plus à $\{0,1,...,N-1\}$ mais bien à l'intervalle [(c'est quoi ces parenthèses malformées ?)](https://fr.wikipedia.org/wiki/Partie_enti%C3%A8re_et_partie_fractionnaire)
 :
$$k\in\left\{-\left\lceil{\frac{N-1}{2}}\right\rceil,\dots,\left\lfloor{\frac{N-1}{2}}\right\rfloor\right\}.$$
A vous de construire correctement ce vecteur des $k$...


In [ ]:
## LE CONTENU DE CETTE CELLLULE EST A SOUMETTRE SUR INGINIOUS

def serieFourier(x,N):
    """
    A COMPLETER
    """
    
    X = np.zeros(N)
    k = np.zeros(N)
    
    return (X,k)


In [ ]:
## CELLULE DE TEST

# Création du signal période x[n], sur une seule période
N = 2
p = 2 # à modifier
n = np.arange(0,p*N)
x  = 2 + np.cos(n*np.pi) # signal de l'exercice 4.6 b)


(X,k) = serieFourier(x,N)

print("k :  ", k)
print("Module de X :", np.abs(X))

# Vous devez normalement obtenir ici la même valeur que les coefficients calculés à l'exercice 4.6 b). Sauf que, suite au
# fftshift, vous obtenez X[-1] et X[0]. Cela étant, les coefficients X[k] étant périodiques de période N=2, on a X[-1]=X[1].

# Testez maintenant de modifier p pour prendre p périodes fondamentales du signal x[n]. 
# Votre fonction remplit-elle les spécifications demandées ? (vecteur X de longueur N ?...)

## 3) Représentation des coefficients en série de Fourier

Finalement, on aimerait représenter les coefficients en série de Fourier sur une figure. Pour cela, nous vous proposons de compléter la fonction `plotSerieFourier(x,X,k,name)` qui crée et sauvegarde une figure contenant trois sous-figures affichant, dans l'ordre : 1) le signal discret $x[n]$ aux indices $n$ appropriés, 2) le graphe en amplitude et 3) le graphe en phase des coefficients $X[k]$ aux indices $k$ appropriés (rappelez-vous, les coefficients $X[k]$ de la série de Fourier peuvent être complexes en toute généralité, d'où l'utilité de créer un graphe en amplitude et en phase). Pour représenter les 3 sous-figures, utilisez la librairie Matplotlib avec la fonction [subplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplot.html). 

Votre fonction sauve également la figure au format png (donc un seul fichier pour les 3 sous-figures) avec le nom *name* (la sauvegarde est déjà implémentée pour vous, ne supprimez pas cette ligne !). Faites attention à respecter la convention d'affichage d'un signal discret (par exemple, en utilisant *plt.stem*), à annoter convenablement vos axes (labels, unités...), et à donner un titre et (si besoin) une légende à vos sous-figures.

In [ ]:
# LE CONTENU DE CETTE CELLLULE EST A SOUMETTRE SUR INGINIOUS

def plotSerieFourier(x,X,k,name):
    """
    A COMPLETER
    """

    # Création de la figure, de taille fixe.
    plt.figure(figsize=(12,16))

    # A MODIFIER : crée(z) ici le plot avec les trois sous-figures
    # par exemple, en partant de ceci :
    #plt.subplot(3,1,1)
    # ...
    #plt.subplot(3,1,2)
    # ...
    #plt.subplot(3,1,3)
    # ...



    plt.subplots_adjust(hspace=0.5) # Pour ajuster l'espace vertical entre sous-figures


    # Sauvegarde de la figure avec le bon nom.
    # Le second argument rétrécit les marges, par défaut relativement larges.
    plt.savefig(name + '.png', bbox_inches='tight')

    return

In [ ]:
## CELLULE DE TEST

# Création du signal période x[n], sur une seule période
N = 24
p = 1 # à modifier
n = np.arange(0,p*N)
x  = 1 + np.sin(n*np.pi/12 + 3*np.pi/8) # signal de l'exercice 4.6 c), vérifiez donc avec ce que vous avez calculé !

(X,k) = serieFourier(x,N)

plotSerieFourier(x,X,k,"test")